In [1]:
import pprint
import pymongo
import json
import connector
from dotenv import load_dotenv
load_dotenv()
import os
from pymongo import MongoClient
user = os.getenv("USER")
password = os.getenv("PASSWORD")
client = pymongo.MongoClient("mongodb+srv://"+ user +":"+ password +"@cluster0-doqes.mongodb.net/test?retryWrites=true&w=majority")
db = connector.connect()
collection_trafic = db.vehicules


In [2]:
print(collection_trafic.find().count())

D:\ProgramData\Anaconda\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


600000


In [3]:
pprint.pprint(list(collection_trafic.find()[0:5]))

[{'_id': ObjectId('5ed3ed8fd59082dd24ba102f'),
  'nb_vehicules': 41,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 2,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba103a'),
  'nb_vehicules': 45,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 14,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba1050'),
  'nb_vehicules': 48,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 18,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba1065'),
  'nb_vehicules': 36,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 57,
  'plage_horaire': 'm'},
 {'_id': ObjectId('5ed3ed8fd59082dd24ba1030'),
  'nb_vehicules': 15,
  'num_arete': 0,
  'num_jour': 0,
  'num_periode': 4,
  'plage_horaire': 'm'}]


In [4]:
collection_trafic.insert_one({"num_arete":501})
print(collection_trafic.find_one({"num_arete":501}))

{'_id': ObjectId('5ef4f1af3226e16e896ab066'), 'num_arete': 501}


In [5]:
collection_trafic.delete_one({"num_arete": 501})
print(collection_trafic.find_one({"num_arete": 501}))

None


In [6]:
vehicules_par_plage = {"m":0, "s":0}
for trafic in collection_trafic.find({"$or":[{"plage_horaire":"m"}, {"plage_horaire":"s"}]}):
    plage = trafic["plage_horaire"]
    vehicules_par_plage[plage] += trafic["nb_vehicules"]
print(vehicules_par_plage)

{'m': 19720247, 's': 28191853}


In [7]:
pipeline = [
    {
        "$group": 
        { 
            "_id": { "plage": "$plage_horaire"}, 
            "total": { "$sum": "$nb_vehicules"} 
        } 
    }
]
vehicules_par_plage = collection_trafic.aggregate(pipeline)
pprint.pprint(list(vehicules_par_plage))

[{'_id': {'plage': 's'}, 'total': 28191853},
 {'_id': {'plage': 'm'}, 'total': 19720247}]


In [8]:
import datetime
from datetime import timedelta, date
print("Hello World")
data_stamped = []

for trafic in collection_trafic.find():
    try :
        jour = trafic['num_jour']
    except KeyError:
        print(trafic)
        continue
    minute = trafic['num_periode']
    plage = trafic['plage_horaire']

    custom_date = datetime.datetime(2020,1,1)

    if plage == 'm' :
        heure = 0
        if minute >= 60 :
            heure = 1
            minute = minute - 60
    custom_date = datetime.datetime(2020,1,1+jour, 7 + heure, minute)

    if plage == 's' :
        heure = 0
        if minute >= 60 :
            heure = 1
            minute = minute - 60
    custom_date = datetime.datetime(2020,1,1+jour, 17 + heure, minute)
    #date_str = str(custom_date)

    row = { "num_arete":trafic['num_arete'],"date":custom_date, "nb_vehicules":trafic['nb_vehicules']}


    data_stamped.append(row)

print(len(data_stamped))

vehicules_stamped = db.vehicules_stamped
vehicules_stamped.delete_many({})

print(db.vehicules_stamped.find().count())

vehicules_stamped.insert_many(data_stamped)

print(db.vehicules_stamped.find().count())

Hello World


ValueError: minute must be in 0..59

In [ ]:
print(db.vehicules_stamped.find().count())

In [ ]:
for row in vehicules_stamped.find() :
    print(row["date"])

In [ ]:
matin = 0
soir = 0

for row in vehicules_stamped.find():
    if row["date"].hour >= 7 and row["date"].hour < 9:
        matin += int(row["nb_vehicules"])
    else:
        soir += int(row["nb_vehicules"])
        
print("Matin: "+ str(matin))
print("Soir: "+ str(soir))